In [1]:
# Kullanılan .csv dosyaları : DIAGNOSES_ICD.csv.gz, D_ICD_DIAGNOSES.csv.gz, NOTEEVENTS.csv.gz
# Amaç: Doktor notlarından hastaya koyulan teşhisin ICD9 kodunu çıkarmak

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string 
import re
from os import listdir
from numpy import array
from collections import Counter
from keras.preprocessing.text import Tokenizer,one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Flatten, Embedding,Input, Dropout,GlobalAveragePooling1D,GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [116]:
def clean_doc(doc):
    tokens = doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('',w) for w in tokens]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word) > 1]
    return tokens
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
def encode_docs(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen = length, padding = 'post')
    return padded


In [6]:
diognosis = pd.read_csv('DIAGNOSES_ICD.csv.gz')
subjects = pd.read_csv('D_ICD_DIAGNOSES.csv.gz')
note_events = pd.read_csv('NOTEEVENTS.csv.gz')

# 3 tane tabloyu dataframe çektim 

/Users/minayilik/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [149]:
sub_diag = pd.merge(diognosis, subjects, on= 'ICD9_CODE')
text_IC9 = pd.merge(sub_diag, note_events, on = 'SUBJECT_ID')
#Key value'larını kullanarak notları ICD9 kodlarını aynı tabloya getirdim 
sub_diag.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,SHORT_TITLE,LONG_TITLE
0,1297,109,172335,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
1,1311,109,173633,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
2,1019,109,131345,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
3,1039,109,131376,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
4,1059,109,135923,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."


In [8]:
columns = text_IC9[["ICD9_CODE", "SHORT_TITLE", "TEXT"]]
text_IC9_df = columns.copy()

#Tabloda kullanmayacağımız columnları düşürdüm

In [504]:
sample_df= text_IC9_df.sample(2000)
sample_df, test_df = train_test_split(sample_df, test_size=0.2)

sample_df = sample_df.reset_index().iloc[:,1:]
test_df = test_df.reset_index().iloc[:,1:]

dat_freq = (sample_df['ICD9_CODE'].value_counts()[:10])

#print(sample_df['TEXT'].value_counts)
sample_df = sample_df.groupby('ICD9_CODE', as_index=False).agg({'TEXT' : ' '.join, 'SHORT_TITLE' : 'first'})
#test_df = test_df.groupby('ICD9_CODE', as_index=False).agg({'TEXT' : ' '.join, 'SHORT_TITLE' : 'first'})
codes = dat_freq.keys()

sample_df = sample_df.loc[sample_df['ICD9_CODE'].isin(codes)]
test_df = test_df.loc[test_df['ICD9_CODE'].isin(codes)]
#bir ICD9 kodu için bir row kalsın diye ICD9 kodları üzerinden group_by yaptım 
#print(sample_df['TEXT'].value_counts)

print(dat_freq)
sample_df 

4280     39
4019     38
42731    34
41401    31
5849     27
53081    23
51881    23
486      20
99592    20
25000    20
Name: ICD9_CODE, dtype: int64


,ICD9_CODE,TEXT,SHORT_TITLE
47,25000,Admission Date: [**2182-7-17**] Discharge...,DMII wo cmp nt st uncntr
164,4019,"Sinus rhythm, rate 66. Since the previous trac...",Hypertension NOS
177,41401,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,Crnry athrscl natve vssl
193,42731,"Chief Complaint:\n HPI:\n 62F with COPD, O...",Atrial fibrillation
198,4280,MICU nursing progress note 7P-7A\nNeuro - A&O ...,CHF NOS
247,486,NURSING PROGRESS NOTE 0700-1900 HOURS:\n** FUL...,"Pneumonia, organism NOS"
264,51881,"62 yo F with LLE cellulitis, R ankle fracture,...",Acute respiratry failure
275,53081,Atrial fibrillation with rapid ventricular res...,Esophageal reflux
330,5849,condition update\nsee carevue for specifics;\n...,Acute kidney failure NOS
499,99592,Neuro: Pt. continues on Propofol & Fentanyl in...,Severe sepsis


In [505]:
#--------------------classification yaklaşımı (çalışmadı) -----------------------------
sample_df['cleaned_Text'] = sample_df.apply(lambda row:clean_doc(row['TEXT']), axis=1)
test_df['cleaned_Text'] = test_df.apply(lambda row:clean_doc(row['TEXT']), axis=1)
# Hasta notlarını kelime kelime ayırdım,temizledim 
sample_df.head()

,ICD9_CODE,TEXT,SHORT_TITLE,cleaned_Text
47,25000,Admission Date: [**2182-7-17**] Discharge...,DMII wo cmp nt st uncntr,"[admission, date, discharge, date, date, birth..."
164,4019,"Sinus rhythm, rate 66. Since the previous trac...",Hypertension NOS,"[sinus, rhythm, rate, since, previous, tracing..."
177,41401,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,Crnry athrscl natve vssl,"[micu, npn, neuro, patient, alert, oriented, f..."
193,42731,"Chief Complaint:\n HPI:\n 62F with COPD, O...",Atrial fibrillation,"[chief, complaint, hpi, copd, osa, obesity, aw..."
198,4280,MICU nursing progress note 7P-7A\nNeuro - A&O ...,CHF NOS,"[micu, nursing, progress, note, neuro, ao, app..."


In [520]:

listy = sample_df['cleaned_Text'].tolist()
ic9 = sample_df['ICD9_CODE'].tolist()
# tüm hasta notlarını docs altında topladım
docs_t = test_df['cleaned_Text'].tolist()
labels_t = test_df['ICD9_CODE'].tolist()

sample_df.head(1000)
#docs = np.array(docs)
#sample_df.drop(['TEXT','SHORT_TITLE'], axis=1, inplace = True)
#sample_df.reset_index(drop=True, inplace=True) 
docs = []
labels=[]

doc_t =[]
label_t =[]

for row in sample_df.values:
    #print(row[0])
    docs.append(row[3])
    labels.append(row[0])
    
for row in sample_df.values:
    #print(row[0])
    doc_t.append(row[3])
    label_t.append(row[0])

print(labels)
label_bef=labels


sample_df

['25000', '4019', '41401', '42731', '4280', '486', '51881', '53081', '5849', '99592']


,ICD9_CODE,TEXT,SHORT_TITLE,cleaned_Text
47,25000,Admission Date: [**2182-7-17**] Discharge...,DMII wo cmp nt st uncntr,"[admission, date, discharge, date, date, birth..."
164,4019,"Sinus rhythm, rate 66. Since the previous trac...",Hypertension NOS,"[sinus, rhythm, rate, since, previous, tracing..."
177,41401,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,Crnry athrscl natve vssl,"[micu, npn, neuro, patient, alert, oriented, f..."
193,42731,"Chief Complaint:\n HPI:\n 62F with COPD, O...",Atrial fibrillation,"[chief, complaint, hpi, copd, osa, obesity, aw..."
198,4280,MICU nursing progress note 7P-7A\nNeuro - A&O ...,CHF NOS,"[micu, nursing, progress, note, neuro, ao, app..."
247,486,NURSING PROGRESS NOTE 0700-1900 HOURS:\n** FUL...,"Pneumonia, organism NOS","[nursing, progress, note, hours, full, code, a..."
264,51881,"62 yo F with LLE cellulitis, R ankle fracture,...",Acute respiratry failure,"[yo, lle, cellulitis, ankle, fracture, respira..."
275,53081,Atrial fibrillation with rapid ventricular res...,Esophageal reflux,"[atrial, fibrillation, rapid, ventricular, res..."
330,5849,condition update\nsee carevue for specifics;\n...,Acute kidney failure NOS,"[condition, update, see, carevue, specifics, n..."
499,99592,Neuro: Pt. continues on Propofol & Fentanyl in...,Severe sepsis,"[neuro, pt, continues, propofol, fentanyl, inf..."


In [521]:
#tokenizer = sample_df.apply(lambda row:create_tokenizer(row['tokenized_Text']), axis=1)
tokenizer = create_tokenizer(docs)
tokenizet_t = create_tokenizer(doc_t)
#tokenizer oluşturdum (oluşturamadım)

In [522]:
#labels = []
#for index,row in sample_df.iterrows():
 #   labels.extend(row['ICD9_CODE'] for _ in range(len(row['cleaned_Text'])))
#label'lları topladım. 
#labels = sample_df['ICD9_CODE'].tolist()


In [539]:

#max_length = max(len(s.split()) for s in docs)
max_length = max(len(text) for text in listy)
max_length_t = max(len(text) for text in docs_t)

vocab_size = len(tokenizer.word_index) +1
vocab_size_t = len(tokenizer_t.word_index)+1

X_train= encode_docs(tokenizer,docs,max_length)
X_test = encode_docs(tokenizer_t,doc_t,max_length_t)
#tokenizer_labels = create_tokenizer(labels)
#labels = encode_docs(tokenizer_labels,labels,1)
le = LabelEncoder()
labels = le.fit_transform(labels)
encoded = to_categorical(labels,num_classes=len(labels)) #one_hot encode
y_train=encoded

label_t = le.fit_transform(label_t)
encoded_t = to_categorical(label_t,num_classes=len(label_t))
y_test = encoded_t

#print(X_train)
#print(X_train.shape)

#vocab lengthi buldum

In [540]:
#X_train.shape
#print(labels)

In [586]:
def define_model2(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length = max_length))
    #model.add(GlobalAveragePooling1D())
    model.add(Conv1D(filters = 32, kernel_size = 8,strides = 1, padding='same', activation = 'relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(10, activation="relu"))
    model.add(Dense(10, activation="softmax"))
    model.add(Flatten())
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    
    return model

In [587]:
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5,5), strides = 1, padding='same', activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same', data_format='channels_last'))
    model.add(Conv2D(64, kernel_size=(5,5), strides = 1,  padding='same', activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same', data_format='channels_last'))
    model.add(Conv2D(128, kernel_size=(5,5), strides = 1, padding='same', activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same', data_format='channels_last'))
    model.add(Flatten())
    model.add(Dense(10, activation="softmax"))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    
    #plot_model(model, to_file = 'model.png', show_shapes = True)
    return model
#model hocanın modeli ile aynı

In [588]:
def define_model_3d(length,vocab_size):
    inputs1 = Input(shape =(length,))
    embedding1 = Embedding(vocab_size, 100) (inputs1)
    conv1 = Conv1D (filters = 32, kernel_size = 4, activation ='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = GlobalMaxPooling1D()(drop1)
    flat1 = Flatten()(pool1)
    
    inputs2 = Input(shape = (length,))
    embedding2 = Embedding(vocab_size,100)(inputs2)
    conv2 = Conv1D(filters = 32, kernel_size = 6, activation ='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = GlobalMaxPooling1D()(drop2)
    flat2 = Flatten()(pool2)
    
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size,100)(inputs3)
    conv3 = Conv1D(filters = 32, kernel_size = 8, activation ='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = GlobalMaxPooling1D()(drop3)
    flat3 = Flatten()(pool3)
    
    merged = concatenate([flat1,flat2,flat3])
    dense1 = Dense(10, activation = 'relu')(merged)
    outputs = Dense(len(labels),activation = 'softmax')(dense1)
    model = Model(inputs = [inputs1,inputs2, inputs3], outputs = outputs)    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    model.summary()
    plot_model(model, show_shapes=True, to_file = 'multichannel.png')
    return model

In [589]:
X_train = np.array(X_train)
y_train = np.array(y_train)
#X=X_train
#y=y_train
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=101)
model = define_model2(vocab_size,max_length)
#print(X_train.shape)
#print(y_train.shape)
model.fit(X_train,y_train, epochs = 7,verbose = 1,batch_size=16)
_,acc = model.evaluate(X_train, y_train, verbose = 0)
print('train accuracy is %.2f' % (acc*100))

#_,acc = model.evaluate(X_test, y_test, verbose = 0)
#print('test accuracy is %.2f' % (acc*100))
#model.save('model.h5')

# ----------------- classification sonu ------------------------------------

Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_76 (Embedding)     (None, 7231, 100)         636000    
_________________________________________________________________
conv1d_74 (Conv1D)           (None, 7231, 32)          25632     
_________________________________________________________________
global_max_pooling1d_74 (Glo (None, 32)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 10)                330       
_________________________________________________________________
dense_118 (Dense)            (None, 10)                110       
_________________________________________________________________
flatten_71 (Flatten)         (None, 10)                0         
Total params: 662,072
Trainable params: 662,072
Non-trainable params: 0
_______________________________________________

In [562]:
len(y_test)

10

define_model_3d

In [590]:
#def predict_IC9(review, vocab, tokenizer, max_length, model):
    #line = clean_doc(review)
    # encode and pad review
    #padded = encode_docs(tokenizer, line, max_length)
  # predict sentiment
    #yhat = model.predict(padded, verbose=2)
    #percent = yhat
    #return percent

#ic9 = predict_IC9(text, docs, tokenizer, max_length, model)


cvp = model.predict_classes(X_train)
print(len(X_test))

for i in range(len(cvp)):
    print("predicted=%s, real =%s" % (cvp[i], y_test[i]))
predictions =[]
#for i in range(len(cvp)):
    #predictions.append(label_bef[index(max(cvp[i]))])
    



10
predicted=0, real =[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
predicted=7, real =[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
predicted=7, real =[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
predicted=0, real =[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
predicted=7, real =[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
predicted=0, real =[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
predicted=0, real =[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
predicted=7, real =[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
predicted=8, real =[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
predicted=9, real =[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [468]:
#test_df = test_df.reset_index().iloc[:,1:] 
#test_df = test_df.loc[sample_df['ICD9_CODE'].isin(codes)]
#test_df['cleaned_Text'] = test_df.apply(lambda row:clean_doc(row['TEXT']), axis=1)



#docs = test_df['cleaned_Text'].tolist()
#labels = test_df['ICD9_CODE'].tolist()

#tokenizer = create_tokenizer(docs)


#max_length = max(len(text) for text in docs)
#vocab_size = len(tokenizer.word_index) +1

#X_test= encode_docs(tokenizer,docs,max_length)

#le = LabelEncoder()
#labels = le.fit_transform(labels)
#encoded = to_categorical(labels,num_classes=len(labels)) #one_hot encode
#y_test= encoded

#X_test = np.array(X_test)
#y_test = np.array(y_test)






IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [466]:
sample_df

,ICD9_CODE,TEXT,SHORT_TITLE,cleaned_Text
7,0389,"Chief Complaint: sepsis, respiratory failure\n...",Septicemia NOS,"[chief, complaint, sepsis, respiratory, failur..."
167,4019,Sinus tachycardia. Lead V6 was not obtained. D...,Hypertension NOS,"[sinus, tachycardia, lead, obtained, delayed, ..."
170,40391,Admission Date: [**2175-1-29**] Dischar...,Hyp kid NOS w cr kid V,"[admission, date, discharge, date, date, birth..."
185,41401,[**2105-4-27**] 1:06 PM\n LUMBO-SACRAL SPINE (...,Crnry athrscl natve vssl,"[pm, lumbosacral, spine, ap, lat, clip, clip, ..."
204,42731,PATIENT/TEST INFORMATION:\nIndication: History...,Atrial fibrillation,"[patienttest, information, indication, history..."
209,4280,[**2102-7-20**] 3:59 AM\n CHEST (PORTABLE AP) ...,CHF NOS,"[chest, portable, ap, clip, clip, number, radi..."
259,486,[**2121-1-9**] 12:40 PM\n CHEST (PORTABLE AP) ...,"Pneumonia, organism NOS","[pm, chest, portable, ap, clip, clip, number, ..."
278,51881,[**2107-7-5**] 4:25 PM\n MR L SPINE W/O CONTRA...,Acute respiratry failure,"[pm, mr, spine, wo, contrast, clip, clip, numb..."
338,5849,Demographics\n Day of intubation:\n Day of...,Acute kidney failure NOS,"[demographics, day, intubation, day, mechanica..."
346,5990,[**2107-3-27**] 7:01 AM\n CHEST (PORTABLE AP) ...,Urin tract infection NOS,"[chest, portable, ap, clip, clip, number, radi..."


In [467]:
test_df

,ICD9_CODE,SHORT_TITLE,TEXT
22080927,71590,Osteoarthros NOS-unspec,[**2168-1-29**] 8:07 PM\n CHEST (PORTABLE AP) ...
27546243,25000,DMII wo cmp nt st uncntr,Impaired Physical Mobility\n Assessment:\n ...
45654655,5307,Mallory-weiss syndrome,[**2173-1-4**] 4:29 PM\n BILAT LOWER EXT VEINS...
50798835,V3000,Single lb in-hosp w/o cs,"[**Location (un) 215**]/NEON DOL 59 , CGA 37 [..."
10086408,43811,Late eff CV dis-aphasia,Respiratory care\npt on the vent tol well no c...
...,...,...,...
48715927,2800,Chr blood loss anemia,MICU Nursing Progress Note- 11a-7p TSICU CAre ...
18590264,5119,Pleural effusion NOS,HPI: Patient is a 70M who is well-known to the...
41620181,59080,Pyelonephritis NOS,[**2192-10-23**] 9:08 AM\n CHEST (PA & LAT) ...
45393739,2536,Neurohypophysis dis NEC,[**2163-4-30**] 1:08 AM\n CHEST (PORTABLE AP) ...
